In [12]:
import os, subprocess

In [4]:
work_dir=os.getcwd()
print(f"Current working directory: {work_dir}")

Current working directory: /opt/app-root/src/stable-diffusion-model


### Apply role and custom roles manually

1. Apply the deployment/role.yaml file to define roles. Update the namespace in the role.yaml file
2. Apply the deployment/role-binding.yaml file to bind role to the service account. Replace the name of the workbench and namespace in the role-binding.yaml file
3. Create a data connection with the name stable-diffusion to point to the s3 bucket
4. Update the tolerations in the deployment/inference-service.yaml


In [10]:
from jinja2 import Template
toleration=os.environ.get('toleration', 'odh-notebook')



In [14]:
# This can take up to 10mins to be ready
# oc get pods or check the kserve-container logs
!oc delete servingruntime stable-diffusion
!oc delete inferenceservice stable-diffusion

template_data = {"toleration": toleration}

def deploy_template(filename, template_data):
    print("invoking template:" + filename)
    template = Template(open(filename).read())
    rendered_template = template.render(template_data)

    subprocess.run(['oc', 'whoami'])
    ps = subprocess.Popen(['echo', rendered_template], stdout=subprocess.PIPE)
    print(ps.stdout)
    output = subprocess.check_output(['oc', 'apply', '-f', '-'], stdin=ps.stdout)
    ps.wait()
    print(f"Deployed template {filename}.")

templates = ['deployment/serving-runtime.yaml','deployment/inference-service.yaml']
for t in templates:
    deploy_template(t, template_data)


servingruntime.serving.kserve.io "stable-diffusion" deleted
Error from server (NotFound): inferenceservices.serving.kserve.io "stable-diffusion" not found
invoking template:deployment/serving-runtime.yaml
system:serviceaccount:worldline:pytorch
<_io.BufferedReader name=72>
Deployed template deployment/serving-runtime.yaml.
invoking template:deployment/inference-service.yaml
system:serviceaccount:worldline:pytorch
<_io.BufferedReader name=72>
Deployed template deployment/inference-service.yaml.


In [ ]:
!until oc get inferenceservice stable-diffusion -o jsonpath='{.status.conditions[?(@.type=="Ready")].status}' | grep -q True ; do \
   echo "Waiting for inference service to be ready..."; \
   sleep 30; \
done

print("Inference serving is ready")

In [ ]:
model_url = !oc get inferenceservice stable-diffusion -o jsonpath='{.status.components.predictor.url}'
model_url = model_url.n

print(f"Model endpoint: {model_url}")

In [ ]:
uniq_token = 'unqtkn'
class_name = 'dog'
my_prompt = f'photo of {uniq_token} {class_name} in a sand castle'

!python {work_dir}/kserve-query.py --url="{model_url}/v1/models/stable-diffusion:predict" --prompt "{my_prompt}" --filename output.jpg

In [ ]:
from IPython.display import Image

Image(filename='output.jpg')